In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"

df_train = pd.read_csv(train_file_path, sep='\t', names=['label', 'message'])
df_test = pd.read_csv(test_file_path, sep='\t', names=['label', 'message'])

df_train['label'] = df_train['label'].map({'ham': 0, 'spam': 1})
df_test['label'] = df_test['label'].map({'ham': 0, 'spam': 1})

In [ ]:
# Separar datos en entrenamiento y prueba
train_messages, train_labels = df_train['message'], df_train['label']
test_messages, test_labels = df_test['message'], df_test['label']

In [ ]:
# Define the vectorize_layer
vectorize_layer = tf.keras.layers.TextVectorization(output_mode='int', output_sequence_length=100)


vectorize_layer.adapt(train_messages.astype(str))

train_messages_vec = vectorize_layer(train_messages.astype(str))
test_messages_vec = vectorize_layer(test_messages.astype(str))

print(train_messages_vec.shape)
print(test_messages_vec.shape)
print(train_labels.shape)
print(test_labels.shape)

In [ ]:
# Construcción del modelo
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=100),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Convertir los mensajes de texto en vectores numéricos
train_messages_vec = vectorize_layer(train_messages)
test_messages_vec = vectorize_layer(test_messages)


In [ ]:
# Convertir las etiquetas a arrays de NumPy
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
# Ajuste para el entrenamiento del modelo
model.fit(train_messages_vec, np.array(train_labels),
          epochs=10, batch_size=32,
          validation_data=(test_messages_vec, np.array(test_labels)))


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    probability = model.predict([pred_text])[0][0]
    label = 'spam' if probability > 0.5 else 'ham'
    return [float(probability), label]

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
     msg_vec = vectorize_layer([msg])
     prediction = predict_message(msg_vec)
     if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
